In [1]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd

### 드라이버 위치

In [28]:
driver = webdriver.Chrome('C:/driver/chromedriver')
#driver = webdriver.Chrome('C:\\Users\\Administrator\\chromedriver')

In [29]:
driver.get('https://manage.searchad.naver.com/customers/1488201/tool/keyword-planner')

### 네이버로그인 누른 뒤 아이디 비번 입력해야될듯

In [25]:
loginbtn=driver.find_element_by_css_selector('#container > div > div > div:nth-child(5) > button > span')
loginbtn.click()

### 아이디\t비번\n

In [24]:
# 여기 로그인 방지문자 나오면 ㅈㅈ
elem=driver.find_element_by_css_selector('#id')
# 아이디\t비번\n
elem.send_keys('jju1183\twjswodnrsla12\n')

### 검색할 키워드
### 키워드\n키워드\n키워드

In [33]:
keyword=driver.find_element_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div:nth-child(1) > div.panel-body > form > div:nth-child(1) > div > div > textarea')
keyword.send_keys('괌롯데호텔\n롯데호텔')


In [34]:
lookupbtn=driver.find_element_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div:nth-child(1) > div.panel-body > form > div.col-xs-12.bg-fill-white > div > div > ul > li > button')
lookupbtn.click()

In [35]:
lookuplist=driver.find_elements_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div.panel-wrap.panel.panel-default > div.widget-wrap.compact-tb.bfx4pn.ng-isolate-scope > div > div > table > tbody > tr')

In [36]:
len(lookuplist) # 1페이지에 키워드 100개 나옴

100

In [38]:
lookuplist[0].find_elements_by_css_selector('td')[1].find_element_by_css_selector('span > span').click()

### 밑에 두칸은 키워드 여러개 검색할 때 창 닫고 다음꺼 클릭하는 부분
### 실제로 쓰려면 for문 맨 뒤에 넣으면 될듯
### 창 크기 작게해서 X가 안보이게 되면 범위에 없다고 나옴
### 최대화 해놓고 다시 실행하면 됨

In [52]:
#closebtn=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-header > div.pull-right.ng-scope > button > i')
#closebtn.click()

In [53]:
#lookuplist[1].find_elements_by_css_selector('td')[1].find_element_by_css_selector('span > span').click()

### id태그 안써도 가능한 지 테스트

In [37]:
# 유동적으로 변하는 id 대신 쭉 타고들어감
driver.find_elements_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-body > div > table > tbody > tr:nth-child(1) > td > div > div > table > tbody > tr > td > div > div > svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-1.highcharts-line-series > path.highcharts-graph')[0].get_attribute('d')
# 원래는 > svg 앞부분이 #highcharts-c85b8hj-0 이딴 식으로 이상하게 변했음 
#driver.find_elements_by_css_selector('#highcharts-c85b8hj-0 > svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-1.highcharts-line-series > path.highcharts-graph')[0].get_attribute('d')

IndexError: list index out of range

![image.png](attachment:image.png)

![image.png](attachment:image.png)

![image.png](attachment:image.png)

![image.png](attachment:image.png)

In [39]:
source=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-body > div > table > tbody > tr:nth-child(1) > td > div > div > table > tbody > tr > td > div > div')

In [40]:
source

<selenium.webdriver.remote.webelement.WebElement (session="a3af616edbcf729c3059c3f5583e149e", element="0.46321846073593687-114")>

### y축 min, max 뽑아옴

In [41]:
#yaxis=driver.find_elements_by_css_selector('#highcharts-97jgmyp-168 > svg > g.highcharts-axis-labels.highcharts-yaxis-labels > text')
#yaxis=driver.find_elements_by_css_selector('#highcharts-97jgmyp-180 > svg > g.highcharts-axis-labels.highcharts-yaxis-labels > text')
yaxis=source.find_elements_by_css_selector('svg > g.highcharts-axis-labels.highcharts-yaxis-labels > text')
yaxis[0].text, yaxis[-1].text

('0', '20k')

In [42]:
minmax=[]
minmax.append(yaxis[0].text)
minmax.append(yaxis[-1].text)

### k -> 000 / 1,000 -> 1000 / 2.5k -> 2.5000 -> 25000 -> 2500

In [43]:
int_minmax=[]
for num in minmax :
    num=num.replace('k','000').replace(',','')
    if '.' in num :
        num=num.replace('.','')
        num=int(num)//10
    int_minmax.append(int(num))

In [44]:
int_minmax

[0, 20000]

### 중간에 g.highcharts-series.highcharts-series-1 부분
- series-0 이 desktop
- series-1 이 mobile

In [45]:
desktop=source.find_element_by_css_selector('svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-0.highcharts-line-series > path.highcharts-graph')
mobile=driver.find_element_by_css_selector('svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-1.highcharts-line-series > path.highcharts-graph')

In [46]:
# 12개월치 x, y좌표가 텍스트로 쭉 나와서 찢어버림
#desktop.get_attribute('d').split(' L')
mobile.get_attribute('d').split(' L')

['M 31.666666666667 93.96',
 ' 95 84.23999999999998',
 ' 158.33333333333 100.44',
 ' 221.66666666667 49.67999999999998',
 ' 285 93.96',
 ' 348.33333333333 77.75999999999999',
 ' 411.66666666667 64.79999999999998',
 ' 475 63.72',
 ' 538.33333333333 59.400000000000006',
 ' 601.66666666667 49.67999999999998',
 ' 665 69.12',
 ' 728.33333333333 108']

In [47]:
d_y_coords=[]
m_y_coords=[]
for line in desktop.get_attribute('d').split(' L') :
    #위에서 찢어놓은 거 맨 뒤 숫자(y좌표)만 가져와
    d_y_coords.append(line.split()[-1])
for line in mobile.get_attribute('d').split(' L') :
    m_y_coords.append(line.split()[-1])

In [48]:
d_y_coords, m_y_coords

(['173.772',
  '170.64',
  '181.548',
  '163.512',
  '177.228',
  '169.452',
  '166.644',
  '164.376',
  '167.184',
  '163.188',
  '169.01999999999998',
  '182.412'],
 ['93.96',
  '84.23999999999998',
  '100.44',
  '49.67999999999998',
  '93.96',
  '77.75999999999999',
  '64.79999999999998',
  '63.72',
  '59.400000000000006',
  '49.67999999999998',
  '69.12',
  '108'])

### ymax - (ymax-ymin)/216 * y좌표 = 검색수

In [49]:
d_searchcount=[]
m_searchcount=[]
for y in d_y_coords :
    d_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/216*float(y))
for y in m_y_coords :
    m_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/216*float(y))

In [50]:
d_searchcount

[3910.0,
 4200.0,
 3190.0,
 4860.0,
 3590.0,
 4310.0,
 4569.999999999998,
 4780.0,
 4520.0,
 4890.0,
 4350.000000000002,
 3110.0]

In [51]:
m_searchcount

[11300.0,
 12200.000000000002,
 10700.0,
 15400.000000000002,
 11300.0,
 12800.0,
 14000.000000000002,
 14100.0,
 14500.0,
 15400.000000000002,
 13600.0,
 10000.0]

In [52]:
desktop_cnt=[]
mobile_cnt=[]
for cnt in d_searchcount :
    #소수점 부분 반올림하고 정수단위로
    desktop_cnt.append(int(np.round(cnt)))
for cnt in m_searchcount :
    mobile_cnt.append(int(np.round(cnt)))

In [53]:
desktop_cnt

[3910, 4200, 3190, 4860, 3590, 4310, 4570, 4780, 4520, 4890, 4350, 3110]

In [54]:
mobile_cnt

[11300,
 12200,
 10700,
 15400,
 11300,
 12800,
 14000,
 14100,
 14500,
 15400,
 13600,
 10000]

In [55]:
total_cnt = []
for i in zip(desktop_cnt, mobile_cnt):
    total_cnt.append(i)

In [56]:
result = pd.DataFrame(total_cnt, columns=['desktop','mobile'], index=[10,11,12,1,2,3,4,5,6,7,8,9])

In [57]:
result

,desktop,mobile
10,3910,11300
11,4200,12200
12,3190,10700
1,4860,15400
2,3590,11300
3,4310,12800
4,4570,14000
5,4780,14100
6,4520,14500
7,4890,15400


In [1]:
# Crome Driver 불러오기
driver = webdriver.Chrome('C:/driver/chromedriver')

# 사이트 이동
driver.get('https://manage.searchad.naver.com/customers/1488201/tool/keyword-planner')


NameError: name 'webdriver' is not defined

---------------------------------

### 최종판

In [1]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd

In [2]:
# Crome Driver 불러오기
driver = webdriver.Chrome('C:/driver/chromedriver')
#driver = webdriver.Chrome('/Users/jaewook/driver/chromedriver')

# 사이트 이동
driver.get('https://manage.searchad.naver.com/customers/1488201/tool/keyword-planner')

In [37]:
#final_result = []
n_keywords = 'adsp자격증'

# 키워드 넣기 - 여기서 for문 
keyword=driver.find_element_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div:nth-child(1) > div.panel-body > form > div:nth-child(1) > div > div > textarea')
keyword.send_keys(n_keywords)

# 조회하기 클릭
lookupbtn=driver.find_element_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div:nth-child(1) > div.panel-body > form > div.col-xs-12.bg-fill-white > div > div > ul > li > button')
lookupbtn.click()

time.sleep(2)

# 키워드 리스트 중 제일 위에 나오는 키워드 클릭
lookuplist=driver.find_elements_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div.panel-wrap.panel.panel-default > div.widget-wrap.compact-tb.bfx4pn.ng-isolate-scope > div > div > table > tbody > tr')

lookuplist[0].find_elements_by_css_selector('td')[1].find_element_by_css_selector('span > span').click()

time.sleep(2)


# 소스 받아오기!
source=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-body > div > table > tbody > tr:nth-child(1) > td > div > div > table > tbody > tr > td > div > div')


# 데이터 정제
yaxis=source.find_elements_by_css_selector('svg > g.highcharts-axis-labels.highcharts-yaxis-labels > text')
yaxis[0].text, yaxis[-1].text

minmax=[]
minmax.append(yaxis[0].text)
minmax.append(yaxis[-1].text)

int_minmax=[]
for num in minmax :
    num=num.replace('k','000').replace(',','')
    if '.' in num :
        num=num.replace('.','')
        num=int(num)//10
    int_minmax.append(int(num))
    
desktop=source.find_element_by_css_selector('svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-0.highcharts-line-series > path.highcharts-graph')
mobile=driver.find_element_by_css_selector('svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-1.highcharts-line-series > path.highcharts-graph')

# 12개월치 x, y좌표가 텍스트로 쭉 나와서 찢어버림
mobile.get_attribute('d').split(' L')


d_y_coords=[]
m_y_coords=[]
for line in desktop.get_attribute('d').split(' L') :
    #위에서 찢어놓은 거 맨 뒤 숫자(y좌표)만 가져와
    d_y_coords.append(line.split()[-1])
for line in mobile.get_attribute('d').split(' L') :
    m_y_coords.append(line.split()[-1])
    
d_searchcount=[]
m_searchcount=[]
for y in d_y_coords :
    d_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/216*float(y))
for y in m_y_coords :
    m_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/216*float(y))

    
desktop_cnt=[]
mobile_cnt=[]
for cnt in d_searchcount :
    #소수점 부분 반올림하고 정수단위로
    desktop_cnt.append(int(np.round(cnt)))
for cnt in m_searchcount :
    mobile_cnt.append(int(np.round(cnt)))


total_cnt = []
for i in zip(desktop_cnt, mobile_cnt):
    total_cnt.append(i)
    
result = pd.DataFrame(total_cnt, columns=['desktop','mobile'], index=[11,12,1,2,3,4,5,6,7,8,9,10])



# 창 끄기
closebtn=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-header > div.pull-right.ng-scope > button > i')
closebtn.click()

keyword.clear()

n_keywords, result.iloc[-1]

('adsp자격증', desktop    1658
 mobile     1331
 Name: 10, dtype: int64)

In [50]:
result.iloc[-1]

desktop     25200
mobile     139000
Name: 10, dtype: int64

In [59]:
final_result = []

In [60]:
final_result.append([n_keywords, desktop_cnt[-1], mobile_cnt[-1]])

In [13]:
final_result

NameError: name 'final_result' is not defined

In [62]:
final_result.append(['일본여행',100,100])

In [63]:
final_result

[['대만여행', 25200, 139000], ['일본여행', 100, 100]]

In [66]:
pd.DataFrame(final_result, columns=['key','desk','mobile'])

,key,desk,mobile
0,대만여행,25200,139000
1,일본여행,100,100


In [7]:
desktop_cnt

[3077, 3711, 3196, 3910, 3365, 4108, 3513, 5425, 5406, 4791, 5188, 5604]

---

In [15]:
kw = pd.read_csv('C:/Users/전재욱/Documents/keywords.csv', engine='python', header=None)
kw

,0
0,괌롯데호텔
1,라세느
2,라세느뷔페
3,라센느
4,롯데라세느
5,롯데리조트속초
6,롯데호텔
7,롯데호텔괌
8,롯데호텔도림
9,롯데호텔라세느


In [23]:
type(kw.iloc[:,0])

pandas.core.series.Series

In [24]:
#keyword_list = ['adsp','adsp강의','빅데이터','머신러닝','롯데호텔','호텔롯데','괌롯데호텔','라세느']
keyword_list = kw.iloc[:,0]

In [25]:
keyword_list

0         괌롯데호텔
1           라세느
2         라세느뷔페
3           라센느
4         롯데라세느
5       롯데리조트속초
6          롯데호텔
7         롯데호텔괌
8        롯데호텔도림
9       롯데호텔라세느
10     롯데호텔레스토랑
11     롯데호텔모모야마
12      롯데호텔무궁화
13       롯데호텔부페
14       롯데호텔뷔페
15      롯데호텔수영장
16       롯데호텔예약
17       롯데호텔울산
18       롯데호텔웨딩
19      롯데호텔패키지
20     롯데호텔페닌슐라
21    롯데호텔프리빌리지
22      롯데호텔하노이
23       롯데호텔할인
24      베트남롯데호텔
25       양곤롯데호텔
26       울산롯데호텔
27      하노이롯데호텔
28      호치민롯데호텔
29         호텔롯데
Name: 0, dtype: object

In [34]:
final_result = []
#n_keywords = 'adsp자격증'

for i in keyword_list:
    # 키워드 넣기 - 여기서 for문 
    keyword=driver.find_element_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div:nth-child(1) > div.panel-body > form > div:nth-child(1) > div > div > textarea')
    keyword.send_keys(i)

    # 조회하기 클릭
    lookupbtn=driver.find_element_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div:nth-child(1) > div.panel-body > form > div.col-xs-12.bg-fill-white > div > div > ul > li > button')
    lookupbtn.click()

    time.sleep(3)

    # 키워드 리스트 중 제일 위에 나오는 키워드 클릭
    lookuplist=driver.find_elements_by_css_selector('#wrap > div > div > div.content > div.page-content-wrapper > div > div > div > div.row > div.col-xs-9.reduce-padding > div.panel-wrap.panel.panel-default > div.widget-wrap.compact-tb.bfx4pn.ng-isolate-scope > div > div > table > tbody > tr')

    lookuplist[0].find_elements_by_css_selector('td')[1].find_element_by_css_selector('span > span').click()

    time.sleep(3)


    # 소스 받아오기!
    source=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-body > div > table > tbody > tr:nth-child(1) > td > div > div > table > tbody > tr > td > div > div')


    # 데이터 정제
    yaxis=source.find_elements_by_css_selector('svg > g.highcharts-axis-labels.highcharts-yaxis-labels > text')
    yaxis[0].text, yaxis[-1].text

    minmax=[]
    minmax.append(yaxis[0].text)
    minmax.append(yaxis[-1].text)

    int_minmax=[]
    for num in minmax :
        num=num.replace('k','000').replace(',','')
        if '.' in num :
            num=num.replace('.','')
            num=int(num)//10
        int_minmax.append(int(num))

    desktop=source.find_element_by_css_selector('svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-0.highcharts-line-series > path.highcharts-graph')
    mobile=driver.find_element_by_css_selector('svg > g.highcharts-series-group > g.highcharts-series.highcharts-series-1.highcharts-line-series > path.highcharts-graph')

    # 12개월치 x, y좌표가 텍스트로 쭉 나와서 찢어버림
    mobile.get_attribute('d').split(' L')


    d_y_coords=[]
    m_y_coords=[]
    for line in desktop.get_attribute('d').split(' L') :
        #위에서 찢어놓은 거 맨 뒤 숫자(y좌표)만 가져와
        d_y_coords.append(line.split()[-1])
    for line in mobile.get_attribute('d').split(' L') :
        m_y_coords.append(line.split()[-1])

    d_searchcount=[]
    m_searchcount=[]
    for y in d_y_coords :
        d_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/216*float(y))
    for y in m_y_coords :
        m_searchcount.append(int_minmax[-1] - (int_minmax[-1]-int_minmax[0])/216*float(y))


    desktop_cnt=[]
    mobile_cnt=[]
    for cnt in d_searchcount :
        #소수점 부분 반올림하고 정수단위로
        desktop_cnt.append(int(np.round(cnt)))
    for cnt in m_searchcount :
        mobile_cnt.append(int(np.round(cnt)))


    #total_cnt = []
    #for i in zip(desktop_cnt, mobile_cnt):
    #    total_cnt.append(i)

    #result = pd.DataFrame(total_cnt, columns=['desktop','mobile'], index=[11,12,1,2,3,4,5,6,7,8,9,10])

    final_result.append([i, desktop_cnt[-1], mobile_cnt[-1]])
    print('"{}" 키워드 정보 수집에 성공했습니다.'.format(i))

    # 창 끄기
    closebtn=driver.find_element_by_css_selector('#wrap > div.modal.tool-keyword-planner-chart-controller.fade.ng-scope.ng-isolate-scope.in > div > div > div > div.modal-header > div.pull-right.ng-scope > button > i')
    closebtn.click()

    keyword.clear()
    time.sleep(2)
#n_keywords, result.iloc[-1]

"괌롯데호텔" 키워드 정보 수집에 성공했습니다.
"라세느" 키워드 정보 수집에 성공했습니다.
"라세느뷔페" 키워드 정보 수집에 성공했습니다.
"라센느" 키워드 정보 수집에 성공했습니다.
"롯데라세느" 키워드 정보 수집에 성공했습니다.
"롯데리조트속초" 키워드 정보 수집에 성공했습니다.
"롯데호텔" 키워드 정보 수집에 성공했습니다.
"롯데호텔괌" 키워드 정보 수집에 성공했습니다.
"롯데호텔도림" 키워드 정보 수집에 성공했습니다.
"롯데호텔라세느" 키워드 정보 수집에 성공했습니다.
"롯데호텔레스토랑" 키워드 정보 수집에 성공했습니다.
"롯데호텔모모야마" 키워드 정보 수집에 성공했습니다.
"롯데호텔무궁화" 키워드 정보 수집에 성공했습니다.
"롯데호텔부페" 키워드 정보 수집에 성공했습니다.
"롯데호텔뷔페" 키워드 정보 수집에 성공했습니다.
"롯데호텔수영장" 키워드 정보 수집에 성공했습니다.
"롯데호텔예약" 키워드 정보 수집에 성공했습니다.
"롯데호텔울산" 키워드 정보 수집에 성공했습니다.
"롯데호텔웨딩" 키워드 정보 수집에 성공했습니다.
"롯데호텔패키지" 키워드 정보 수집에 성공했습니다.
"롯데호텔페닌슐라" 키워드 정보 수집에 성공했습니다.
"롯데호텔프리빌리지" 키워드 정보 수집에 성공했습니다.
"롯데호텔하노이" 키워드 정보 수집에 성공했습니다.
"롯데호텔할인" 키워드 정보 수집에 성공했습니다.
"베트남롯데호텔" 키워드 정보 수집에 성공했습니다.
"양곤롯데호텔" 키워드 정보 수집에 성공했습니다.
"울산롯데호텔" 키워드 정보 수집에 성공했습니다.
"하노이롯데호텔" 키워드 정보 수집에 성공했습니다.
"호치민롯데호텔" 키워드 정보 수집에 성공했습니다.
"호텔롯데" 키워드 정보 수집에 성공했습니다.


In [35]:
df = pd.DataFrame(final_result, columns=['keywords','desk','mobile'])
df

,keywords,desk,mobile
0,괌롯데호텔,4070,11700
1,라세느,5780,16300
2,라세느뷔페,1070,3820
3,라센느,1040,3790
4,롯데라세느,1700,5020
5,롯데리조트속초,4450,15100
6,롯데호텔,23800,40100
7,롯데호텔괌,890,1720
8,롯데호텔도림,960,3000
9,롯데호텔라세느,11800,47800


In [33]:
pd.DataFrame(final_result, columns=['keywords','desk','mobile']).sort_values(by=['desk'],ascending=False)

,keywords,desk,mobile
6,롯데호텔,23800,40100
9,롯데호텔라세느,11800,47800
1,라세느,5780,16300
5,롯데리조트속초,4450,15100
0,괌롯데호텔,4070,11700
14,롯데호텔뷔페,2190,8630
26,울산롯데호텔,1860,5320
4,롯데라세느,1700,5020
27,하노이롯데호텔,1250,3560
29,호텔롯데,1200,1070


In [36]:
df.to_csv('C:/Users/전재욱/Documents/nkeywords.csv', encoding='euc_kr')